In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
import os
import pandas as pd

In [3]:
 url = "http://www.espn.com/mlb/war/leaders/_/type/seasonal/year/20"

In [4]:
year_list = ['19', '18', '17', '16', '15']
df_list = []

In [5]:
for year in year_list:
    query_url = url+year
    qdfl = pd.read_html(query_url)
    qdfl[0]['season'] = year
    df_list.append(qdfl[0])

In [12]:
data = pd.concat(df_list)
data.head()

,0,1,2,3,4,5,6,7,8,9,season
0,2019 Season WAR Leaders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,RK,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,1,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,2,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,3,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [16]:
cols = ['rank', 'player', 'WAR', 'OFF', 'DEF', 'WAA', 'TRPG', 'ORPG', 'RAA', 'WAAWP', 'Season']

In [19]:
data.columns = cols

data

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19
6,Justin Verlander,7.8,0,0,5.8,5,3.4,56.6,0.7,19
7,Mike Minor,7.8,0,0,5.9,5,3.2,57,0.7,19
8,Lance Lynn,7.6,0,0,5.7,5,3.3,55.6,0.7,19
9,Jacob deGrom,7.3,0,0,5.6,4.7,3.1,50.2,0.7,19
